In [87]:
from pyspark.sql import SparkSession, functions as F, types as T
from pyspark.sql.window import Window
from pyspark import SparkConf
from dotenv import load_dotenv
import os

In [91]:
query = """
CREATE OR REPLACE TABLE NY_TAXI.STAGE.OBT_TRIPS AS
SELECT
    -- tiempo
    PICKUP_DATETIME,
    DROPOFF_DATETIME,
    TO_DATE(PICKUP_DATETIME) AS PICKUP_DATE,
    EXTRACT(HOUR FROM PICKUP_DATETIME) AS PICKUP_HOUR,
    TO_DATE(DROPOFF_DATETIME) AS DROPOFF_DATE,
    EXTRACT(HOUR FROM DROPOFF_DATETIME) AS DROPOFF_HOUR,
    DAYOFWEEK(PICKUP_DATETIME) AS DAY_OF_WEEK,
    MONTH(PICKUP_DATETIME) AS MONTH,
    YEAR(PICKUP_DATETIME) AS YEAR,

    -- ubicacion
    PULOCATIONID AS PU_LOCATION_ID,
    PICKUP_ZONE AS PU_ZONE,
    PICKUP_BOROUGH AS PU_BOROUGH,
    DOLOCATIONID AS DO_LOCATION_ID,
    DROPOFF_ZONE AS DO_ZONE,
    DROPOFF_BOROUGH AS DO_BOROUGH,
    
    -- servicios y codigos
    SERVICE_TYPE,
    VENDORID AS VENDOR_ID,
    VENDORID_DESC AS VENDOR_NAME,
    RATECODEID AS RATE_CODE_ID,
    RATECODE_DESC AS RATE_CODE_DESC,
    PAYMENT_TYPE,
    PAYMENT_TYPE_DESC,
    TRIP_TYPE,
    TRIP_TYPE_DESC,

    -- viaje
    PASSENGER_COUNT,
    TRIP_DISTANCE,
    STORE_AND_FWD_FLAG_DESC AS STORE_AND_FWD_FLAG,
    
    -- tarifas
    FARE_AMOUNT,
    EXTRA,
    MTA_TAX,
    TIP_AMOUNT,
    TOLLS_AMOUNT,
    IMPROVEMENT_SURCHARGE,
    CONGESTION_SURCHARGE,
    AIRPORT_FEE,
    TOTAL_AMOUNT,

    -- derivadas
    TRIP_DURATION_MINUTES AS TRIP_DURATION_MIN,
    CASE 
        WHEN TRIP_DURATION_MINUTES > 0 
        THEN (TRIP_DISTANCE / (TRIP_DURATION_MINUTES / 60)) 
        ELSE NULL 
    END AS AVG_SPEED_MPH,
    CASE 
        WHEN TOTAL_AMOUNT > 0 
        THEN (TIP_AMOUNT / TOTAL_AMOUNT) * 100 
        ELSE NULL 
    END AS TIP_PCT,

    -- lineage
    RUN_ID,
    INGESTED_AT_UTC,
    SERVICE_TYPE AS SOURCE_SERVICE,
    SOURCE_YEAR,
    SOURCE_MONTH
FROM NY_TAXI.RAW.TAXI_TRIPS_ENRICHED;
"""

In [92]:
import snowflake.connector

conn = snowflake.connector.connect(
    #Se tiene que cambiar a los secretosAAAAAAA
    user="MARE122510",
    password="MyTurnEra2025100%",
    account="TLZAPUN-PKC06603",
    warehouse="COMPUTE_WH",
    database="NY_TAXI",
    schema="STAGE",
    role =os.getenv('ROLE')
)

cur = conn.cursor()
print("Cursor creado")

cur.execute(query)
print("Tabla OBT_TRIPs creada exitosamente")

cur.close()
conn.close()


Cursor creado
Tabla OBT_TRIPs creada exitosamente
